In [ ]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_matrix_multiplication")' SchwiftyNotebook_matrix_multiplication

In [2]:
//export
import Path
import TensorFlow
import SchwiftyNotebook_matrix_multiplication

In [3]:
let thingo = Tensor([-2, -1, 0, 1, 2])

In [4]:
max(thingo, 0)

[0.0, 0.0, 0.0, 1.0, 2.0]


In [5]:
//export
public func reLU<T>(tensor: Tensor<T>) -> Tensor<T> where T : FloatingPoint, T: TensorFlowScalar {
    return max(tensor, 0)
}

In [6]:
//export
public func linearCombination<T>(inputs: Tensor<T>, weights: Tensor<T>, bias: Tensor<T>) -> Tensor<T> where T: FloatingPoint, T: TensorFlowScalar {
    return matmul(inputs, weights) + bias
}

In [7]:
let (xTrainingData, yTrainingData, xValidationData, yValidationData) = loadMNISTData(path: mnistPath)

In [34]:
let xTrainingDataNormalized = normalizeTensor(tensor: xTrainingData)
                                        .reshaped(to: [xTrainingData.shape[0], 784])
let xValidationDataNormalized = normalizeTensor(tensor: xValidationData)
                                        .reshaped(to: [xValidationData.shape[0], 784])

In [9]:
// export
public typealias TensorFloat=Tensor<Float>
public func assertNearZero(_ tensor: TensorFloat, _ threshold: Float = 1e-3){
    assert(tensor < threshold, "Expected \(tensor) to be less than \(threshold)")
}

In [35]:
assertNearZero(xTrainingDataNormalized.mean())
assertNearZero(xValidationDataNormalized.mean())
assertNearZero(1 - xTrainingDataNormalized.standardDeviation())
assertNearZero(1 - xValidationDataNormalized.standardDeviation())

In [36]:
let (numberOfImages, numberOfPixels) = (xTrainingDataNormalized.shape[0], xTrainingDataNormalized.shape[1])
let numberOfClasses = 10
let layerOutput = 75

In [37]:
let parameterLayerOne = TensorFloat(randomNormal: [numberOfPixels, layerOutput]) / sqrt(Float(numberOfImages))
let parameterLayerTwo = TensorFloat(randomNormal: [layerOutput, 1]) / sqrt(Float(layerOutput))

In [38]:
assertNearZero(parameterLayerOne.mean())
assertNearZero(parameterLayerOne.standardDeviation() - 1 / sqrt(Float(numberOfImages)))
// assertNearZero(parameterLayerTwo.mean())
// assertNearZero(parameterLayerTwo.standardDeviation() - 1 / sqrt(Float(layerOutput)))

In [52]:
let biasLayerOne = TensorFloat(zeros: [layerOutput])
let biasLayerTwo = TensorFloat(zeros: [1])

In [53]:
assertNearZero(biasLayerOne.mean())
assertNearZero(biasLayerOne.standardDeviation() - 1 / sqrt(Float(numberOfImages)))

In [54]:
print(xValidationDataNormalized.shape, parameterLayerOne.shape, biasLayerOne.shape)

[10000, 784] [784, 75] [75]


In [64]:
import Dispatch

// ⏰Time how long it takes to run the specified function, optionally taking
// the average across a number of repetitions.
public func withTime<T>(_ f: () -> T) -> T {
    let start = DispatchTime.now()
    let value = f()
    let end = DispatchTime.now()
    let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
    let milliseconds = nanoseconds / 1e6
    print("elapsed time: \(milliseconds) ms")
    return value
}

In [85]:
let linearWomboCombo = withTime {
    withDevice(.gpu){
        linearCombination(inputs: xValidationDataNormalized, 
                                         weights: parameterLayerOne, 
                                         bias: biasLayerOne)
    }
}

Fatal error: Device /job:localhost/replica:0/task:0/device:GPU:0 not found: file /swift-base/tensorflow-swift-apis/Sources/TensorFlow/Core/Runtime.swift, line 1030
Current stack trace:
0    libswiftCore.so                    0x00007f58cdbd58b0 swift_reportError + 50
1    libswiftCore.so                    0x00007f58cdc44aa0 _swift_stdlib_reportFatalErrorInFile + 115
2    libswiftCore.so                    0x00007f58cdb6cace <unavailable> + 3738318
3    libswiftCore.so                    0x00007f58cdb6cc47 <unavailable> + 3738695
4    libswiftCore.so                    0x00007f58cd93ac4d <unavailable> + 1436749
5    libswiftCore.so                    0x00007f58cdb41a78 <unavailable> + 3562104
6    libswiftCore.so                    0x00007f58cd93a0a9 <unavailable> + 1433769
7    libswiftTensorFlow.so              0x00007f58c9d6af73 <unavailable> + 2514803
8    libswiftTensorFlow.so              0x00007f58c9d6a986 <unavailable> + 2513286
9    libswiftTensorFlow.so              0x00007f58

: 

In [71]:
linearWomboCombo.shape //that ain't falco

▿ [60000, 75]
  ▿ dimensions : 2 elements
    - 0 : 60000
    - 1 : 75


In [73]:
let activations = withTime { reLU(tensor: linearWomboCombo) }

elapsed time: 355.649171 ms


In [57]:
activations.shape

▿ [10000, 75]
  ▿ dimensions : 2 elements
    - 0 : 10000
    - 1 : 75


In [75]:
print(activations.mean(), activations.standardDeviation())

0.042250283 0.06293498


In [74]:
let otherLinearWomboCombo = withTime {linearCombination(inputs: activations, weights: parameterLayerTwo, bias: biasLayerTwo)}
otherLinearWomboCombo.shape

elapsed time: 3.733576 ms


▿ [60000, 1]
  ▿ dimensions : 2 elements
    - 0 : 60000
    - 1 : 1


0.08791636
